In [ ]:
%load_ext autoreload
%autoreload 2

In [28]:
import time
import logging
from typing import List, Dict, Any

from news_summarizer.utils import batch
from transformers import pipeline
from news_summarizer.domain.clean_documents import CleanedArticle
from transformers import AutoModelForCausalLM, AutoTokenizer
from news_summarizer.domain.prompt import GenerateDatasetSamplesPrompt
from news_summarizer.datasets.generation import SummarizationDatasetGenerator

logger = logging.getLogger()
logger.setLevel(logging.INFO)


In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct-AWQ"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [30]:
system_prompt_template = """Você é um assistente especializado em resumir notícias
"""


summarization_prompt_template = """Resuma essa noticia. Para isso você deve: Ler toda a notícia. \
Determinar o tema central da notícia e as informações mais relevantes que o autor deseja transmitir. \
Focar nos fatos e dados mais importantes, deixando de lado detalhes secundários ou exemplos específicos. \
Evitar adicionar opiniões pessoais ou interpretações. O resumo deve refletir fielmente o conteúdo original. \
Escrever de forma direta e simples, evitando jargões ou termos técnicos desnecessários. \
Manter a ordem lógica das informações apresentadas no artigo original, garantindo que o resumo seja coeso e fácil de entender. \ 
Certificar-se de que todas as informações incluídas no resumo estão corretas e correspondem ao conteúdo do artigo. \

Você deve responder utilizando a mesma lingua da notícia. \
Cada resposta deve conter apenas o resumo, sem palavras chave como 'Resumo:', não deve possuir multiplos espaços entre os parágrafos.
Noticia
{article}"""

In [31]:
def fetch_all_documents():
    offset = None
    my_documents = []

    while True:
        documents, offset = CleanedArticle.bulk_find(**{}, offset=offset)
        my_documents.extend(documents)

        if offset is None:
            break

    return my_documents

def get_prompt(document: CleanedArticle) -> GenerateDatasetSamplesPrompt:
    input_variables = {'article': document.content}
    prompt = summarization_prompt_template.format(**input_variables)
    prompt_tokens = tokenizer.encode(prompt)
    num_tokens = len(prompt_tokens)

    generated_dataset_sample = GenerateDatasetSamplesPrompt(
        template=summarization_prompt_template,
        input_variables=input_variables,
        content=prompt,
        num_tokens=num_tokens,
        data_category='summariation_dataset',
        document=document,
    )

    return generated_dataset_sample

def get_prompts(documents: list[CleanedArticle]) -> list[GenerateDatasetSamplesPrompt]:
    return [get_prompt(document) for document in documents]

In [ ]:
documents = fetch_all_documents()
prompts = get_prompts(documents)

In [ ]:
generator = SummarizationDatasetGenerator()
generator.generate(prompts)